In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import tensorflow as tf
import glob
import sys
import tempfile
# you shouldn't need to make any more imports

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[np.arange(y.shape[0]),y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)
    

In [4]:
def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.05)
  return tf.Variable(initial)


def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def norm(x): 
  return tf.nn.lrn(x, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

In [5]:
def deepnn(x):
  """deepnn builds the graph for a deep net for classifying digits.
  Args:
    x: an input tensor with the dimensions (N_examples, 3072), where 3072 is the
    number of pixels in a standard CIFAR image.
  Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
  """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 32, 32, 3])
    #x_image = tf.image.per_image_standardization(x_image)

  # First convolutional layer - maps one grayscale image to 64 feature maps.
  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 5, 3, 64])
    b_conv1 = bias_variable([64])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
  with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)
    
  # First Normalization layer.
  with tf.name_scope('norm1'):
    h_norm1 = norm(h_pool1)

  # Second convolutional layer -- maps 64 feature maps to 64.
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 64, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_norm1, W_conv2) + b_conv2)
    
  # Second Normalization layer.
  with tf.name_scope('norm2'):
    h_norm2 = norm(h_conv2)
    
  # Second pooling layer.
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_norm2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 32x32x3 image
  # is down to 8x8x64 feature maps -- maps this to 512 features.
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([8*8*64, 512])
    b_fc1 = bias_variable([512])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([512, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    
    
    
  #Set up some logging stuff
  trainables = tf.trainable_variables()
  #Make histograms of WEIGHTS
  for i, Variable in enumerate(trainables):
    tf.summary.histogram(Variable.name,Variable)  
    
  return y_conv, keep_prob


In [6]:
# Load the data, using utility functions from HW1
data_root_path = 'cifar10-hw1/'
X, Y = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test').astype(np.float32)
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [7]:
def Get_PreprocessInput(X_train):
    """
    Do standard scaling;
    --or do whitening or PCA on the data.
        
    Assumes X is (Nfeatures x Nexamples) numpy array
    """
    mu = X_train.mean(axis=1)
    sigma = X_train.std(axis=1)#mu*0.+1.#X_train.std(axis=1)#X_train.var(axis=1)
    return mu, sigma

    
def Do_PreprocessInput(X, mu, sigma):
    """
    Do the preprocessing on the given data, 
    given some already calculated mu + sigma
    which come from the training data.
    """
    X -= mu.reshape(X.shape[0],1).repeat(X.shape[1],axis=1)
    epsilon = 10e-9
    X /= (sigma.reshape(X.shape[0],1).repeat(X.shape[1],axis=1)+epsilon)
    return X

In [8]:
mu, sigma = Get_PreprocessInput(X)
X = Do_PreprocessInput(X, mu, sigma)
X_test = Do_PreprocessInput(X_test, mu, sigma)

In [9]:
# Hold out 10% of the data to use as a validation set:
#y_OH = tf.one_hot(Y,10)
train_inds = np.random.choice(X.shape[1],int(X.shape[1]*.90),replace=False)
validation_inds = np.setdiff1d(np.arange(X.shape[1]),train_inds)
X_train = X[:,train_inds].astype(np.float32)
y_train = Y[train_inds].astype(np.int32)
X_validation = X[:,validation_inds].astype(np.float32)
y_validation = Y[validation_inds].astype(np.int32)
#X_placeholder = tf.placeholder(X_train.dtype, X_train.shape)
#y_placeholder = tf.placeholder(y_train.dtype, y_train.shape)
#dataset_train = tf.contrib.data.Dataset.from_tensor_slices((X_placeholder, y_placeholder))
#iterator = dataset_train.make_initializable_iterator()
#sess.run(iterator.initializer, feed_dict={X_placeholder: X_train,
                                          #y_placeholder: tfy_train})

In [10]:
def get_batch(X, y, batch_size):
    selector = np.random.choice(y.shape[1], batch_size, replace=False)
    return X[:, selector].T, y[:,selector].T

In [11]:
x = tf.placeholder(tf.float32, [None, 3072])
y_ = tf.placeholder(tf.float32, [None, 10])

In [12]:
y_conv, keep_prob = deepnn(x)

INFO:tensorflow:Summary name conv1/Variable:0 is illegal; using conv1/Variable_0 instead.
INFO:tensorflow:Summary name conv1/Variable_1:0 is illegal; using conv1/Variable_1_0 instead.
INFO:tensorflow:Summary name conv2/Variable:0 is illegal; using conv2/Variable_0 instead.
INFO:tensorflow:Summary name conv2/Variable_1:0 is illegal; using conv2/Variable_1_0 instead.
INFO:tensorflow:Summary name fc1/Variable:0 is illegal; using fc1/Variable_0 instead.
INFO:tensorflow:Summary name fc1/Variable_1:0 is illegal; using fc1/Variable_1_0 instead.
INFO:tensorflow:Summary name fc2/Variable:0 is illegal; using fc2/Variable_0 instead.
INFO:tensorflow:Summary name fc2/Variable_1:0 is illegal; using fc2/Variable_1_0 instead.


In [13]:
with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)

In [14]:
cross_entropy = tf.reduce_mean(cross_entropy)

In [15]:
OPTIMIZER = 'ADAM'#'SGD'

#with tf.name_scope('adam_optimizer'):
#    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
global_step = tf.contrib.framework.get_or_create_global_step()
with tf.name_scope('learning_rate'):
    lr = tf.train.exponential_decay(0.1,
                                  global_step,
                                  450*350,
                                  0.1,
                                  staircase=True)

if OPTIMIZER=='SGD':
    with tf.name_scope('gradientDescent_optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(lr)
        train_step = optimizer.minimize(cross_entropy)
if OPTIMIZER=='ADAM':
    with tf.name_scope('ADAM_optimizer'):
        #Don't need to specify lr, ADAM determines ~optimal lr
        optimizer = tf.train.AdamOptimizer()
        train_step = optimizer.minimize(cross_entropy)



In [16]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

In [17]:
y_predicted = tf.argmax(y_conv, 1)

In [18]:
#

In [19]:
with tf.Session() as sess:
    
    graph_location = tempfile.mkdtemp()
    #train_writer = tf.summary.FileWriter(graph_location)
    #train_writer.add_graph(tf.get_default_graph())
    print('Saving graph to: %s' % graph_location)
    train_writer = tf.summary.FileWriter(graph_location,sess.graph)
    
    
    if OPTIMIZER=='ADAM':
      tf.summary.scalar('train__learning_rate', optimizer._lr)
        
    if OPTIMIZER=='SGD':
      tf.summary.scalar('train__learning_rate', optimizer._learning_rate)
        
    #Make histograms of last2 dense layers weights GRADIENTS:
    trainables = tf.trainable_variables()

    #Make hists of gradients
    for i, Variable in enumerate(trainables):
      tf.summary.histogram(Variable.name+'__grad',optimizer.compute_gradients(cross_entropy, Variable))

    #Add loss, accuracy to tensorboard:
    tf.summary.scalar('train__loss', cross_entropy)

    tf.train.SummarySaverHook(save_steps=50, output_dir=graph_location,
                          summary_op=tf.summary.merge_all, summary_writer=train_writer)
    
    
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
      batch_xs, batch_ys = get_batch(X_train, one_hot(y_train), 100)
      if i % 100 == 0:   
        train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0}) 
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.4})

    print('validation accuracy %g' % accuracy.eval(feed_dict={
        x: X_validation.T, y_: one_hot(y_validation).T, keep_prob: 1.0}))
    save_predictions('ans1-uni', y_predicted.eval(feed_dict={x: X_test.T, keep_prob: 1.0})) 

Saving graph to: /var/folders/0h/2s3xs7_d3mq83xk38vh_gbnh0000gn/T/tmpxvbawz9w
INFO:tensorflow:Summary name conv1/Variable:0__grad is illegal; using conv1/Variable_0__grad instead.
INFO:tensorflow:Summary name conv1/Variable_1:0__grad is illegal; using conv1/Variable_1_0__grad instead.
INFO:tensorflow:Summary name conv2/Variable:0__grad is illegal; using conv2/Variable_0__grad instead.
INFO:tensorflow:Summary name conv2/Variable_1:0__grad is illegal; using conv2/Variable_1_0__grad instead.
INFO:tensorflow:Summary name fc1/Variable:0__grad is illegal; using fc1/Variable_0__grad instead.
INFO:tensorflow:Summary name fc1/Variable_1:0__grad is illegal; using fc1/Variable_1_0__grad instead.
INFO:tensorflow:Summary name fc2/Variable:0__grad is illegal; using fc2/Variable_0__grad instead.
INFO:tensorflow:Summary name fc2/Variable_1:0__grad is illegal; using fc2/Variable_1_0__grad instead.
step 0, training accuracy 0.07
step 100, training accuracy 0.38
step 200, training accuracy 0.45
step 300,

In [20]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

(10000,)


array([3, 1, 0, 4, 5, 0, 8, 4, 8, 1])

In [21]:
tf.reset_default_graph()